 ## Exercise 49 - UDF

In [14]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

inputPath = "/data/students/bigdata-01QYD/ex_data/Ex49/data"
outputPathV1 = "res_out_Ex49/"

__input__ : 
* A CSV file containing a list of user profiles : _name, age, gender_

__output__ : 
*  A CSV file containing one line for each profile. The original age attribute is substituted with a new attributed called rangeage of type String : 
    * rangeage = "[" + (age/10)*10 + "-" + (age/10)*10 +9"]"

### DataFrames

In [17]:
# map into (movieid , userid)
df = spark.read.load(inputPath, format="csv", header=True, inferSchema=True)
df.show()

+-----+-------+---+
| name|surname|age|
+-----+-------+---+
|Paolo|  Garza| 42|
| Luca| Boccia| 41|
|Maura|Bianchi| 16|
+-----+-------+---+



In [19]:
spark.udf.register("AgeBinning", lambda age: "["+str((age//10)*10)+"-"+str((age//10)*10+9)+"]")

<function __main__.<lambda>(age)>

In [20]:
df_binned = df.selectExpr("name", "surname", "AgeBinning(age) as rangeage")

In [21]:
df_binned.show()

+-----+-------+--------+
| name|surname|rangeage|
+-----+-------+--------+
|Paolo|  Garza| [40-49]|
| Luca| Boccia| [40-49]|
|Maura|Bianchi| [10-19]|
+-----+-------+--------+



### SQL

In [24]:
df.createOrReplaceTempView("people")

In [26]:
df_sql_binned = spark.sql("""
                             SELECT name, surname, AgeBinning(age) as binned_age
                             FROM people
                          """)
df_sql_binned.show()

+-----+-------+----------+
| name|surname|binned_age|
+-----+-------+----------+
|Paolo|  Garza|   [40-49]|
| Luca| Boccia|   [40-49]|
|Maura|Bianchi|   [10-19]|
+-----+-------+----------+

